In [1]:
import numpy as np
import h5py
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras import regularizers

Using TensorFlow backend.


In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) 
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) 

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) 
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) 

    classes = np.array(test_dataset["list_classes"][:]) 
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [3]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.
print(Y_train_orig[0][0],Y_train_orig[0][1])
# Reshape
#Y_train = to_categorical(Y_train_orig, num_classes=len(classes), dtype='float32').reshape((X_train.shape[0],len(classes)))
#Y_test = to_categorical(Y_test_orig, num_classes=len(classes), dtype='float32').reshape((X_test.shape[0],len(classes)))
Y_train=Y_train_orig.T
Y_test=Y_test_orig.T
print(Y_train.shape)

0 0
(209, 1)


In [4]:
def catClassifierModel(input_shape):
    X_input=Input(input_shape)
    X=Conv2D(8,(4,4),strides=(1,1),padding='same',name='conv0',kernel_regularizer=regularizers.l2(0.01))(X_input)
    X=BatchNormalization(axis=3,name='bn0')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((8,8),name='max_pool0',padding='same')(X)
    X=Conv2D(16,(2,2),strides=(1,1),padding='same',name='conv1',kernel_regularizer=regularizers.l2(0.01))(X)
    X=BatchNormalization(axis=3,name='bn1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((4,4),name='max_pool1',padding='same')(X)
    X=Flatten()(X)
    X=Dense(1,activation='sigmoid')(X)
    model=Model(inputs = X_input, outputs = X, name='catClassifier')
    
    return model

In [5]:
model = catClassifierModel((64,64,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 8)         392       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 8)         0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 8, 8, 8)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 8, 8, 16)          528       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 8, 8, 16)          64        
__________

In [6]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [7]:
model.fit(x=X_train, y=Y_train , epochs=100, batch_size=64)

Epoch 1/100
209/209 [==============================] - 4s 18ms/step - loss: 0.9770 - acc: 0.5215
Epoch 2/100
209/209 [==============================] - 0s 582us/step - loss: 0.8521 - acc: 0.6029
Epoch 3/100
209/209 [==============================] - 0s 601us/step - loss: 0.7854 - acc: 0.6555
Epoch 4/100
209/209 [==============================] - 0s 591us/step - loss: 0.7482 - acc: 0.6651
Epoch 5/100
209/209 [==============================] - 0s 596us/step - loss: 0.7112 - acc: 0.6746
Epoch 6/100
209/209 [==============================] - 0s 595us/step - loss: 0.6929 - acc: 0.7033
Epoch 7/100
209/209 [==============================] - 0s 625us/step - loss: 0.6690 - acc: 0.7129
Epoch 8/100
209/209 [==============================] - 0s 554us/step - loss: 0.6597 - acc: 0.7177
Epoch 9/100
209/209 [==============================] - 0s 673us/step - loss: 0.6373 - acc: 0.7225
Epoch 10/100
209/209 [==============================] - 0s 740us/step - loss: 0.6267 - acc: 0.7368
Epoch 11/100
209/209

In [8]:
preds = model.evaluate(x=X_test,y=Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

50/50 [==============================] - 0s 4ms/step

Loss = 0.4053150153160095
Test Accuracy = 0.8800000047683716
